# PyTorch - CNN

In [1]:
import torch
import torch.nn as nn #defining neural network
import torch.nn.functional as F #helper
from torch.utils.data import DataLoader #batching
from torchvision import datasets, transforms #loading datasets
from torchvision.utils import make_grid  #for visualization

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [8]:
# !pip install torchvision
import torchvision
torchvision.__version__

'0.16.0'

## 1. Load the dataset

In [4]:
#define some transforms (convert 0-255 to 0-1)
transform = transforms.ToTensor()
#if you want to crop the image, rotate the image; they are all done in this transform

#use pytorch datasets to load MNIST - dataset of digit images
#train
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)

#validation set
train_set, val_set = torch.utils.data.random_split(train_data, [50000, 10000])

#test
test_data  = datasets.MNIST(root='data', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:01<00:00, 7008603.91it/s] 


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 35513249.44it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3662657.10it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7257344.29it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



## 2. Define the network

## 3. Define the loss and optimizer

## 4. Train!!

## 5. Testing

## 6. Analysis

## 7. Inference